# Normal

In [ ]:
import torch
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('uitnlp/visobert')
model = XLMRobertaForSequenceClassification.from_pretrained('/home4/bachpt/text_classification/Visobert/saved_checkpoints/checkpoint-50170')
model.eval()
model.to('cuda')

In [ ]:
# import torch
# from transformers import AutoTokenizer, RobertaForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
# model = RobertaForSequenceClassification.from_pretrained('/home4/bachpt/text_classification/phobert/saved_checkpoints/checkpoint-50170')

In [ ]:
import pandas as pd
data = pd.read_csv('toxic_test.csv')
data['result'] = ''
data.head()

In [ ]:
logits

In [ ]:
model.config.id2label = {0: 'normal', 1: 'negative'}

In [ ]:
model.config.id2label[predicted_class_id]

In [13]:
# test individual sentences
import torch.nn.functional as F

# inputs = tokenizer("những người nào xây dựng nên chatbot này", return_tensors="pt", truncation=True, max_length=256).to('cuda')
inputs = tokenizer("cái đám mất_dạy bắc kì thì hay quá gọi họ là cái gì hả bạn?",
                    return_tensors="pt", truncation=True, max_length=512).to('cuda')

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
softmax_values = F.softmax(logits, dim=1)
print(softmax_values)

# if predicted_class_id == 1:        
        # if softmax_values[0][1] < 0.7:
            # predicted_class_id = 0

# print(logits)
model.config.id2label = {0: 'Yeah', 1: 'Nope'}
model.config.id2label[predicted_class_id]

tensor([[5.1965e-06, 9.9999e-01]], device='cuda:0')


'Nope'

In [4]:
# Viso save the result for test set
# frozen = 0.04906 (CPU), 0.0084 (GPU)
# unfrozen = 0.0507 (CPU), 0.00805 (GPU)

import torch
# import time
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('uitnlp/visobert')
model = XLMRobertaForSequenceClassification.from_pretrained('/home4/bachpt/text_classification/Visobert/saved_checkpoints/checkpoint-50170')

model.cuda()
model.eval()

import pandas as pd
data = pd.read_csv('/home4/bachpt/text_classification/mixed_test_lowered_shuffled.csv')
data['predicted label'] = ''
data['softmax'] = ''

import torch.nn.functional as F

data_text = data['text']
# total_time = 0
predictions = []
confidences = []
for datum in data_text:
    inputs = tokenizer(datum, return_tensors="pt", truncation=True, max_length=512).to('cuda')
    with torch.no_grad():
        # start_time = time.time()
        logits = model(**inputs).logits
        # end_time = time.time()

    predicted_class_id = logits.argmax().item()
    # softmax_value = F.softmax(logits, dim=1)
    
    # if predicted_class_id == 1:
        # if softmax_values[0][1] < 0.7:
            # predicted_class_id = 0

    # Save the predicted_class_id to the column result
    predictions.append(predicted_class_id)
    # confidences.append(softmax_value)
    # inference_time = end_time - start_time
    # total_time += inference_time

data['predicted label'] = predictions
# data['softmax'] = confidences
# average_time = total_time / len(data_text)
# print("Average Inference Time per Sample:", average_time)
data.to_csv('Viso_mixed_test_result_lowered_shuffled.csv', index = False)

# To get GPU usage
# Uncomment the following lines if you are using GPU
# import torch.cuda as cuda
# gpu_usage = cuda.max_memory_allocated()
# print("GPU Memory Usage:", gpu_usage)

In [5]:
# Pho save the result for test set
# no segment = 0.0496 (CPU), 0.0084 (GPU)
# segment = 0.0433(CPU), 0.0082 (GPU)

import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification
import time
# import torch.nn.functional as F
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
model = RobertaForSequenceClassification.from_pretrained('/home4/bachpt/text_classification/Phobert_segment/saved_checkpoints/checkpoint-50170')

model.cuda()
model.eval()

data = pd.read_csv('/home4/bachpt/text_classification/mixed_test_lowered_shuffled_segmented.csv')

data_text = data['text']
total_time = 0
predictions = []
for datum in data_text:
    inputs = tokenizer(datum, return_tensors="pt", truncation=True, max_length=256).to('cuda')
    with torch.no_grad():
        start_time = time.time()
        logits = model(**inputs).logits
        end_time = time.time()

    predicted_class_id = logits.argmax().item()
    # softmax_values = F.softmax(logits, dim=1)
    
    # if predicted_class_id == 1:
        # if softmax_values[0][1] < 0.7:
            # predicted_class_id = 0

    # Save the predicted_class_id to the column result
    predictions.append(predicted_class_id)
    inference_time = end_time - start_time
    total_time += inference_time
data['predicted label'] = predictions
average_time = total_time / len(data_text)
print("Average Inference Time per Sample:", average_time)
data.to_csv('Frozen_Pho_mixed_segmented_test_result_lowered_shuffled.csv', index = False)

# To get GPU usage
# Uncomment the following lines if you are using GPU
# import torch.cuda as cuda
# gpu_usage = cuda.max_memory_allocated()
# print("GPU Memory Usage:", gpu_usage)

Average Inference Time per Sample: 0.007972929934917142


In [4]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
model = RobertaForSequenceClassification.from_pretrained('/home4/bachpt/text_classification/Phobert_segment/saved_checkpoints/checkpoint-50170')
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

# ONNX

## Test single sentence

In [ ]:
# Convert to ONNX format
# !optimum-cli export onnx --model /home4/bachpt/text_classification/Visobert/saved_checkpoints/checkpoint-50170 --monolith --task text-classification model.onnx

In [ ]:
import onnx
onnx_model = onnx.load('/home4/bachpt/text_classification/ONNX_Viso/model.onnx/model.onnx')
onnx.checker.check_model(onnx_model)

In [ ]:
from optimum.pipelines import pipeline
classifier = pipeline(task = 'text-classification', model = model, tokenizer = tokenizer, accelerator = 'ort')

In [ ]:
classifier('nơi nào có thể xem biểu mẫu học tập')

## Models

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('uitnlp/visobert')
tokenizer.save_pretrained('/home4/bachpt/text_classification/ONNX_Viso/model.onnx')

In [ ]:
# onnxruntime
# numpy

In [ ]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification
from optimum.pipelines import pipeline

tokenizer = AutoTokenizer.from_pretrained('/home4/bachpt/text_classification/ONNX_Viso/model')

# Loading directly an ONNX model from a model repo.
model = ORTModelForSequenceClassification.from_pretrained("/home4/bachpt/text_classification/ONNX_Viso/model.onnx")

onnx_cl = pipeline("text-classification", model=model, tokenizer=tokenizer, accelerator="ort")

import pandas as pd
from tqdm import tqdm
data = pd.read_csv('/home4/bachpt/text_classification/mixed_test_lowered_shuffled.csv')
data['predicted label'] = ''
data_text = data['text']
predictions = []

for datum in tqdm(data_text):
    pred = onnx_cl(datum)
    predictions.append(pred)

data['predicted label'] = predictions
data.to_csv('Viso_ONNX_mixed_test_result_lowered_shuffled.csv', index = False)

In [ ]:
# model.cuda()
# model.eval()
import time

import pandas as pd
data = pd.read_csv('/home4/bachpt/text_classification/mixed_test_lowered_shuffled.csv')
data['predicted label'] = ''

# import torch.nn.functional as F

data_text = data['text']
total_time = 0
predictions = []
for datum in data_text:
    inputs = tokenizer(datum, return_tensors="pt", truncation=True, max_length=512)#.to('cuda')
    with torch.no_grad():
        start_time = time.time()
        logits = model(**inputs).logits
        end_time = time.time()

    predicted_class_id = logits.argmax().item()
    # softmax_values = F.softmax(logits, dim=1)
    
    # if predicted_class_id == 1:
        # if softmax_values[0][1] < 0.7:
            # predicted_class_id = 0

    # Save the predicted_class_id to the column result
    predictions.append(predicted_class_id)
    inference_time = end_time - start_time
    total_time += inference_time
data['predicted label'] = predictions
average_time = total_time / len(data_text)
print("Average Inference Time per Sample:", average_time)
data.to_csv('Viso_ONNX_mixed_test_result_lowered_shuffled.csv', index = False)

# To get GPU usage
# Uncomment the following lines if you are using GPU
# import torch.cuda as cuda
# gpu_usage = cuda.max_memory_allocated()
# print("GPU Memory Usage:", gpu_usage)